# Dependencies

In [16]:
# !pip install pandas
# !pip install transformers
# !pip install --upgrade jupyter ipywidgets
# !pip install torch torchvision torchaudio
# !pip install tensorflow
# !pip install flax

# Import data

In [56]:
import pandas as pd

data = pd.read_csv("../assign1/datasets/training_data.csv")
small_data = data.sample(100)
small_data

,text,label
12621,"Northwest charges for in-person, phone ticket ...",2
63881,Home Builder Optimism Reported Rising (Reuters...,2
24364,AL Wrap: Yankees Stunned by Orioles Onslaught ...,1
75537,AMD Geode Personal Internet Communicator launc...,3
85013,"Germany, 15 Years After the Wall Today, German...",0
...,...,...
84955,Tesco Launches Online Music Store (NewsFactor)...,3
49689,Major League Baseball Sets Attendance Record H...,1
2915,Russia #39;s Korzhanenko wins women #39;s shot...,1
21682,"Tkachuk sends #39;em packin #39; So much, it ...",1


In [57]:
label_dict = {
    0: 'World', 
    1: 'Sports', 
    2: 'Business',
    3: 'Sci/Tech'
}

# Transformer

In [63]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline

# Load the model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("dstefa/roberta-base_topic_classification_nyt_news")
model = AutoModelForSequenceClassification.from_pretrained("dstefa/roberta-base_topic_classification_nyt_news")

# Load the text classification pipeline
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer)

# Apply the pipeline to get predictions
small_data['predictions'] = small_data['text'].apply(lambda x: pipe(x)[0]['label'])

In [64]:
predictions_dict = {
    "Arts, Culture, and Entertainment": 0,
    "Health and Wellness": 0,
    "Lifestyle and Fashion": 0,
    "Politics": 0,
    "Crime": 0,
    "Sports": 1,
    "Business and Finance": 2,
    "Science and Technology": 3
}

In [65]:
# Transform prediction to integers using label_dict
small_data['predictions_label'] = small_data['predictions'].apply(lambda x: predictions_dict[x])

# Compare predictions with actual label
comparison = small_data['predictions_label'] == small_data['label']

# Print comparison results
print(comparison)

12621     True
63881     True
24364     True
75537    False
85013    False
         ...  
84955    False
49689     True
2915      True
21682     True
93182     True
Length: 100, dtype: bool


In [66]:
accuracy = comparison.mean()
print(f'Accuracy: {accuracy * 100}%')

Accuracy: 72.0%


In [67]:
small_data['predictions_label'].describe()

count    100.000000
mean       1.230000
std        0.951925
min        0.000000
25%        0.000000
50%        1.000000
75%        2.000000
max        3.000000
Name: predictions_label, dtype: float64